In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import sys
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground import *
import bartz
import arviz as az

INFO:arviz.preview:arviz_base not installed
INFO:arviz.preview:arviz_stats not installed
INFO:arviz.preview:arviz_plots not installed


In [3]:
proposal_probs = {"multi_grow": 0.25, "multi_prune": 0.25, "multi_change": 0.4, "multi_swap": 0.1}
#proposal_probs = {"multi_grow": 0.5, "multi_prune": 0.5}
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

bart = MultiBART(ndpost=50, nskip=20, n_trees=100, proposal_probs=proposal_probs, multi_tries=1)
bart.fit(X_train, y_train)

profiler.stop()
profiler.print()

Iterations: 100%|██████████| 70/70 [00:16<00:00,  4.27it/s]



  _     ._   __/__   _ _  _  _ _/_   Recorded: 01:19:51  Samples:  12323
 /_//_/// /_\ / //_// / //_'/ //     Duration: 16.405    CPU time: 16.031
/   _/                      v5.0.1

Profile at C:\Windows\Temp\ipykernel_23180\2353391020.py:4

16.405 <module>  C:\Windows\Temp\ipykernel_23180\2353391020.py:1
└─ 16.405 MultiBART.fit  bart_playground\bart.py:27
   └─ 16.403 MultiSampler.run  bart_playground\samplers.py:83
      └─ 16.340 MultiSampler.one_iter  bart_playground\samplers.py:358
         ├─ 11.817 MultiGrow.propose  bart_playground\moves.py:33
         │  ├─ 7.132 MultiGrow.try_propose  bart_playground\moves.py:180
         │  │  ├─ 4.936 BARTLikelihood.trees_log_marginal_lkhd  bart_playground\priors.py:279
         │  │  │  ├─ 3.859 CPUDispatcher._compile_for_args  numba\core\dispatcher.py:344
         │  │  │  │     [205 frames hidden]  numba, llvmlite
         │  │  │  └─ 0.930 Parameters.leaf_basis  bart_playground\params.py:675
         │  │  │     └─ 0.927 Tree.leaf_bas

In [5]:
rf = RandomForestRegressor(random_state=42)
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=50, nskip=20)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

INFO:2025-06-24 01:20:10,623:jax._src.xla_bridge:927: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-06-24 01:20:10,626:jax._src.xla_bridge:927: Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.


In [6]:
profiler = Profiler()
profiler.start()

default_proposal_probs = {"grow": 0.25, "prune": 0.25, "change": 0.4, "swap": 0.1}
bart_default = DefaultBART(ndpost=50, nskip=20, n_trees=100, proposal_probs=default_proposal_probs)
bart_default.fit(X_train, y_train)

profiler.stop()
profiler.print()

Iterations: 100%|██████████| 70/70 [00:03<00:00, 22.42it/s]



  _     ._   __/__   _ _  _  _ _/_   Recorded: 01:20:17  Samples:  3090
 /_//_/// /_\ / //_// / //_'/ //     Duration: 3.132     CPU time: 3.031
/   _/                      v5.0.1

Profile at C:\Windows\Temp\ipykernel_23180\3672608610.py:2

3.135 <module>  C:\Windows\Temp\ipykernel_23180\3672608610.py:1
└─ 3.135 DefaultBART.fit  bart_playground\bart.py:27
   └─ 3.133 DefaultSampler.run  bart_playground\samplers.py:83
      └─ 3.097 DefaultSampler.one_iter  bart_playground\samplers.py:280
         ├─ 1.979 Swap.propose  bart_playground\moves.py:33
         │  ├─ 1.008 Swap.try_propose  bart_playground\moves.py:146
         │  │  ├─ 0.847 Tree.swap_split  bart_playground\params.py:398
         │  │  │  └─ 0.831 Tree.change_split  bart_playground\params.py:391
         │  │  │     └─ 0.819 Tree.update_n  bart_playground\params.py:405
         │  │  │        ├─ 0.436 Tree.update_n  bart_playground\params.py:405
         │  │  │        │  ├─ 0.247 [self]  bart_playground\params.py
        

In [7]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz,
          "bart_default" : bart_default}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.02112037030574617,
 'rf': 0.022139023845392215,
 'lr': 0.048045521328019404,
 'btz': 0.022409798535525886,
 'bart_default': 0.02292141162635982}